![Timeout Effectiveness by Season](DSA-210-PROJECT/dsa%20project/outputs/figures/effectiveness_by_season_bar.png)
![Run Termination by Season](DSA-210-PROJECT/dsa%20project/outputs/figures/run_termination_by_season.png)

The bar chart shows timeout effectiveness rates across different NBA seasons. Interestingly, the effectiveness of timeouts has remained relatively consistent over time (around 56–59%), with the 1999-00 season showing slightly higher effectiveness at 58.8%.

This consistency suggests that despite changes in playing style and pace across NBA eras, the fundamental effectiveness of timeouts in disrupting opponent momentum has remained stable. This supports the idea that the psychological and strategic benefits of timeouts transcend specific eras of basketball.

The stacked bar chart shows whether opponent scoring runs continued or were terminated after timeouts across different seasons. Consistent with the effectiveness rates, run termination rates have remained relatively stable across seasons, ranging from about 56% to 59%.

The consistency in both effectiveness rates and run termination rates across different eras of NBA basketball suggests that timeouts serve a fundamental psychological and strategic purpose that transcends changes in playing style and rules.


### 5. Timeout Effectiveness by Scoring Run Size

Do timeouts become more effective when the opponent is on a larger scoring run? Let's analyze timeout effectiveness based on the size of the opponent's scoring run.

In [ ]:
# Run size analysis
if 'run_size_bin' in df.columns:
    run_size_analysis = df.groupby('run_size_bin').agg({
        'effective': ['count', 'sum', 'mean'],
        'efficiency_change': 'mean'
    }).reset_index()
    
    run_size_analysis.columns = ['Run_Size', 'Count', 'Effective_Count', 'Effectiveness_Rate', 'Avg_Change']
    
    # Drop NaN values if any
    run_size_analysis = run_size_analysis.dropna(subset=['Run_Size'])
    
    # Create bar chart of effectiveness by run size
    plt.figure(figsize=(12, 8))
    bars = plt.bar(run_size_analysis['Run_Size'], run_size_analysis['Effectiveness_Rate'], 
                 color='skyblue', edgecolor='navy')
    
    # Add data labels on top of each bar
    for i, bar in enumerate(bars):
        count = run_size_analysis.iloc[i]['Count']
        pct = run_size_analysis.iloc[i]['Effectiveness_Rate'] * 100
        avg_change = run_size_analysis.iloc[i]['Avg_Change']
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
               f"n={count}\n{pct:.1f}%\nΔ={avg_change:.3f}", ha='center', va='bottom', fontsize=10)
    
    # Add horizontal line for overall effectiveness
    overall_effectiveness = df['effective'].mean()
    plt.axhline(y=overall_effectiveness, color='red', linestyle='--', 
              label=f'Overall: {overall_effectiveness:.1%}')
    
    # Format y-axis as percentage
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
    
    plt.xlabel('Opponent Scoring Run Size (Points)', fontsize=14)
    plt.ylabel('Timeout Effectiveness Rate\n(% of Timeouts that Reduced Opponent Efficiency)', fontsize=14)
    plt.title('Timeout Effectiveness Rate by Opponent Scoring Run Size', fontsize=16)
    plt.ylim(0, max(run_size_analysis['Effectiveness_Rate']) * 1.2)  # Add some space for labels
    plt.legend()
    plt.savefig('effectiveness_by_run_size_bar.png', dpi=300, bbox_inches='tight')
    plt.tight_layout()
    plt.show()

print(f"\nTimeout Effectiveness: {effective_rate:.1%} of timeouts effectively reduced opponent offensive efficiency")
print(f"Run Termination: {termination_rate:.1%} of opponent scoring runs were terminated after timeouts")
    
    # Create boxplot of efficiency change by run size
    plt.figure(figsize=(12, 8))
    
    # Get unique run sizes and filter out empty ones
    run_sizes = sorted([size for size in df['run_size_bin'].unique() if pd.notna(size)])
    
    # Filter data to only include valid run sizes
    filtered_df = df[df['run_size_bin'].isin(run_sizes)]
    
    # Create boxplot with filtered data
    ax = sns.boxplot(x='run_size_bin', y='efficiency_change', data=filtered_df, order=run_sizes)
    
    # Add sample size and mean information to each run size
    for i, run_size in enumerate(run_sizes):
        run_data = filtered_df[filtered_df['run_size_bin'] == run_size]
        count = len(run_data)
        mean = run_data['efficiency_change'].mean()
        
        # Add text above each box
        plt.text(i, plt.ylim()[1]*0.9, f"n = {count}\nMean = {mean:.3f}", 
               ha='center', va='top', fontsize=11,
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    plt.xlabel('Opponent Scoring Run Size (Points)', fontsize=14)
    plt.ylabel('Change in Offensive Efficiency\n(Post-Timeout minus Pre-Timeout)\nPoints Per Possession', fontsize=14)
    plt.title('Offensive Efficiency Change by Opponent Scoring Run Size', fontsize=16)
    
    # Adjust figure size to remove excess whitespace
    plt.xlim(-0.5, len(run_sizes)-0.5)
    plt.savefig('efficiency_by_run_size_boxplot.png', dpi=300, bbox_inches='tight')
    plt.tight_layout()
    plt.show()
else:
    print("Run size data is not available in the dataset.")

![Effectiveness by Run Size](DSA-210-PROJECT/dsa project/outputs/figures/effectiveness_by_run_size_bar.png)
![Efficiency Change by Run Size](DSA-210-PROJECT/dsa project/outputs/figures/efficiency_by_run_size_boxplot.png)

The analysis reveals an interesting pattern: timeouts appear to be more effective at disrupting opponent momentum when called during larger scoring runs. For 8-9 point runs, timeouts were effective 76.1% of the time, compared to 56.7% for 6-7 point runs and 66.7% for 10-11 point runs.

This suggests that timeouts may be particularly valuable when the opponent has built significant momentum (8-9 point runs), providing coaches with a powerful tool to disrupt momentum at critical junctures. The boxplot further illustrates this pattern, showing larger negative changes in offensive efficiency for 8-9 and 10-11 point runs compared to 6-7 point runs.

However, it's important to note the sample size differences - there are many more 6-7 point runs than larger runs, which is expected given the relative rarity of longer scoring runs in NBA games.

### 6. Timeout Effectiveness by Game Quarter

Does the effectiveness of timeouts vary based on when they are called during the game? Let's analyze timeout effectiveness by quarter.

In [ ]:
# Quarter analysis
quarter_analysis = df.groupby('quarter').agg({
    'effective': ['count', 'sum', 'mean'],
    'efficiency_change': 'mean',
    'run_terminated': 'mean'
}).reset_index()

quarter_analysis.columns = ['Quarter', 'Count', 'Effective_Count', 'Effectiveness_Rate', 'Avg_Change', 'Termination_Rate']

# Sort quarters chronologically
quarter_order = {'Q1': 0, 'Q2': 1, 'Q3': 2, 'Q4': 3, 'OT1': 4, 'OT2': 5, 'OT3': 6}
quarter_analysis['Order'] = quarter_analysis['Quarter'].map(quarter_order)
quarter_analysis = quarter_analysis.sort_values('Order')

# Create boxplot of efficiency change by quarter
plt.figure(figsize=(12, 8))

# Create boxplot
sorted_quarters = quarter_analysis['Quarter'].tolist()
ax = sns.boxplot(x='quarter', y='efficiency_change', data=df, order=sorted_quarters)

# Add sample size and mean information to each quarter
for i, quarter in enumerate(sorted_quarters):
    quarter_data = df[df['quarter'] == quarter]
    count = len(quarter_data)
    mean = quarter_data['efficiency_change'].mean()
    
    # Add text above each box
    plt.text(i, plt.ylim()[1]*0.9, f"n = {count}\nMean = {mean:.3f}", 
             ha='center', va='top', fontsize=11,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.xlabel('Game Quarter', fontsize=14)
plt.ylabel('Change in Offensive Efficiency\n(Post-Timeout minus Pre-Timeout)\nPoints Per Possession', fontsize=14)
plt.title('Offensive Efficiency Change by Quarter', fontsize=16)
plt.savefig('efficiency_by_quarter_boxplot.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

# Create bar chart of run termination by quarter
plt.figure(figsize=(12, 8))

# Create bar chart
bars = plt.bar(quarter_analysis['Quarter'], quarter_analysis['Termination_Rate'], 
             color='skyblue', edgecolor='navy')

# Add data labels on top of each bar
for i, bar in enumerate(bars):
    count = quarter_analysis.iloc[i]['Count']
    pct = quarter_analysis.iloc[i]['Termination_Rate'] * 100
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
           f"n={count}\n{pct:.1f}%", ha='center', va='bottom', fontsize=10)

# Add horizontal line for overall run termination rate
overall_rate = df['run_terminated'].mean()
plt.axhline(y=overall_rate, color='red', linestyle='--', 
          label=f'Overall: {overall_rate:.1%}')

# Format y-axis as percentage
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

plt.xlabel('Game Quarter', fontsize=14)
plt.ylabel('Run Termination Rate\n(% of Opponent Scoring Runs Stopped After Timeout)', fontsize=14)
plt.title('Opponent Scoring Run Termination Rate by Quarter', fontsize=16)
plt.ylim(0, max(quarter_analysis['Termination_Rate']) * 1.2)  # Add some space for labels
plt.legend()
plt.savefig('run_termination_by_quarter.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

![Efficiency Change by Quarter](DSA-210-PROJECT/dsa project/outputs/figures/efficiency_by_quarter_boxplot.png)
![Run Termination by Quarter](DSA-210-PROJECT/dsa project/outputs/figures/run_termination_by_quarter.png)

The analysis reveals interesting patterns in timeout effectiveness across different game periods:

1. First quarter (Q1) timeouts show the highest effectiveness (66.1% effective with mean change of -7.5 points per possession), suggesting that early interventions to disrupt opponent momentum can be particularly valuable.

2. Timeout effectiveness gradually decreases as the game progresses, with fourth quarter (Q4) timeouts showing the lowest effectiveness (54.0%).

3. The larger efficiency changes in Q1 compared to later quarters suggest that teams may be more responsive to strategic adjustments early in the game, while late-game momentum might be harder to disrupt.

This pattern could be explained by various factors including player fatigue, psychological pressure in close late-game situations, or teams having already made their primary adjustments by the later stages of the game.

### 7. Timeout Effectiveness by NBA Team

Do some NBA teams use timeouts more effectively than others? Let's examine team-specific timeout effectiveness rates.

In [ ]:
# Team analysis
team_analysis = df.groupby('opponent_abbr').agg({
    'effective': ['count', 'sum', 'mean'],
    'efficiency_change': 'mean'
}).reset_index()

team_analysis.columns = ['Team', 'Count', 'Effective_Count', 'Effectiveness_Rate', 'Avg_Change']
team_analysis = team_analysis[team_analysis['Count'] >= 10]  # Filter to teams with sufficient data
team_analysis = team_analysis.sort_values('Effectiveness_Rate', ascending=False).head(15)

# Create bar chart of team effectiveness
plt.figure(figsize=(14, 10))

# Create bar chart
bars = plt.bar(team_analysis['Team'], team_analysis['Effectiveness_Rate'], 
              color='skyblue', edgecolor='navy')

# Add data labels on top of each bar
for i, bar in enumerate(bars):
    count = team_analysis.iloc[i]['Count']
    pct = team_analysis.iloc[i]['Effectiveness_Rate'] * 100
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f"n={count}\n{pct:.1f}%", ha='center', va='bottom', fontsize=10)

# Add horizontal line for overall effectiveness
overall_effectiveness = df['effective'].mean()
plt.axhline(y=overall_effectiveness, color='red', linestyle='--', 
           label=f'Overall: {overall_effectiveness:.1%}')

# Format y-axis as percentage
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

plt.xlabel('NBA Team Calling Timeout', fontsize=14)
plt.ylabel('Timeout Effectiveness Rate\n(% of Timeouts that Reduced Opponent Efficiency)', fontsize=14)
plt.title('Timeout Effectiveness Rate by NBA Team (Top 15)', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, max(team_analysis['Effectiveness_Rate']) * 1.2)  # Add some space for labels
plt.legend()
plt.savefig('effectiveness_by_team_bar.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

![Effectiveness by Team](DSA-210-PROJECT/dsa project/outputs/figures/effectiveness_by_team_bar.png)

The analysis reveals significant variation in timeout effectiveness across NBA teams:

1. Orlando (ORL), New Orleans (NOP), and Detroit (DET) demonstrate the highest timeout effectiveness rates (72.0%, 70.5%, and 68.0% respectively).

2. All teams in the top 15 show timeout effectiveness rates above the overall average of 57.4%.

These differences could reflect various factors:
- Coaching strategies and the specific adjustments made during timeouts
- Team composition and player responsiveness to coaching
- Opponent tendencies and preparation

The variation suggests that while timeouts are generally effective across the league, some coaching staffs may be more adept at using timeouts strategically to disrupt opponent momentum.

### 8. Pre vs Post Timeout Shooting Efficiency

Let's examine how specific aspects of offensive efficiency change after timeouts, focusing on field goal percentage and true shooting percentage.

In [ ]:
# Create comparison of shooting metrics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# 1. Field Goal Percentage Comparison
pre_fg = df['pre_timeout_fg_pct'].mean()
post_fg = df['post_timeout_fg_pct'].mean()

# Calculate statistics
t_stat_fg, p_val_fg = stats.ttest_rel(df['pre_timeout_fg_pct'], df['post_timeout_fg_pct'])

# Format p-value using scientific notation for very small values
if p_val_fg < 1e-4:
    p_val_fg_str = f"{p_val_fg:.2e}"
else:
    p_val_fg_str = f"{p_val_fg:.4f}"

# Create bar chart
x = ['Pre-Timeout', 'Post-Timeout']
y = [pre_fg, post_fg]

ax1.bar(x, y, color=['blue', 'red'], alpha=0.7)

# Add data labels
ax1.text(0, pre_fg+0.01, f"{pre_fg:.1%}", ha='center', fontsize=12)
ax1.text(1, post_fg+0.01, f"{post_fg:.1%}", ha='center', fontsize=12)

# Add statistical test results
if p_val_fg < 0.05:
    significance = "Significant difference"
else:
    significance = "Not significant"
    
ax1.text(0.5, max(y)*1.1, f"p={p_val_fg_str}\n{significance}", 
       ha='center', fontsize=12,
       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Format y-axis as percentage
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

ax1.set_ylim(0, max(y)*1.2)  # Add some space for labels
ax1.set_title('Field Goal Percentage Comparison', fontsize=14)
ax1.set_ylabel('Field Goal Percentage', fontsize=12)

# 2. True Shooting Percentage Comparison
pre_ts = df['pre_timeout_ts'].mean()
post_ts = df['post_timeout_ts'].mean()

# Calculate statistics
t_stat_ts, p_val_ts = stats.ttest_rel(df['pre_timeout_ts'], df['post_timeout_ts'])

# Format p-value using scientific notation for very small values
if p_val_ts < 1e-4:
    p_val_ts_str = f"{p_val_ts:.2e}"
else:
    p_val_ts_str = f"{p_val_ts:.4f}"

# Create bar chart
y = [pre_ts, post_ts]

ax2.bar(x, y, color=['blue', 'red'], alpha=0.7)

# Add data labels
ax2.text(0, pre_ts+0.01, f"{pre_ts:.1%}", ha='center', fontsize=12)
ax2.text(1, post_ts+0.01, f"{post_ts:.1%}", ha='center', fontsize=12)

# Add statistical test results
if p_val_ts < 0.05:
    significance = "Significant difference"
else:
    significance = "Not significant"
    
ax2.text(0.5, max(y)*1.1, f"p={p_val_ts_str}\n{significance}", 
       ha='center', fontsize=12,
       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Format y-axis as percentage
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

ax2.set_ylim(0, max(y)*1.2)  # Add some space for labels
ax2.set_title('True Shooting Percentage Comparison', fontsize=14)
ax2.set_ylabel('True Shooting Percentage', fontsize=12)

plt.suptitle('Opponent Shooting Efficiency: Pre-Timeout vs Post-Timeout', fontsize=16)
plt.savefig('pre_post_shooting_comparison.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

![Shooting Efficiency Comparison](DSA-210-PROJECT/dsa project/outputs/figures/pre_post_shooting_comparison.png)

The analysis reveals dramatic changes in shooting efficiency after timeouts:

1. **Field Goal Percentage** drops from 55.9% before timeouts to 41.2% after timeouts, a 14.7 percentage point decrease.

2. **True Shooting Percentage** (which accounts for three-pointers and free throws) drops from 63.7% to 51.1%, a 12.6 percentage point decrease.

Both changes are highly statistically significant (p < 0.001), providing strong evidence that timeouts disrupt the opponent's shooting rhythm.

This dramatic decrease in shooting efficiency suggests that a primary mechanism by which timeouts disrupt momentum is breaking the opponent's shooting rhythm. Whether through defensive adjustments, psychological effects, or simply interrupting the flow of the game, timeouts appear to be particularly effective at reducing an opponent's shooting effectiveness.

### 9. Overall Timeout Effectiveness Visualization

Let's visualize the overall effectiveness of timeouts in a simple pie chart to provide a clear summary of our findings.

In [ ]:
# Create pie chart of overall timeout effectiveness
plt.figure(figsize=(10, 10))

# Calculate effectiveness statistics
effective_count = int(df['effective'].sum())
ineffective_count = len(df) - effective_count
effective_rate = df['effective'].mean()
ineffective_rate = 1 - effective_rate

# Create data for the pie chart
labels = [f'Effective\n{effective_count} timeouts ({effective_rate:.1%})', 
         f'Ineffective\n{ineffective_count} timeouts ({ineffective_rate:.1%})']
sizes = [effective_rate * 100, ineffective_rate * 100]
colors = ['green', 'red']
explode = (0.1, 0)  # Explode the first slice

# Create pie chart
patches, texts, autotexts = plt.pie(sizes, explode=explode, labels=labels, colors=colors,
                                 autopct='%1.1f%%', shadow=True, startangle=90,
                                 textprops={'fontsize': 14, 'fontweight': 'bold'})

# Equal aspect ratio ensures the pie chart is circular
plt.axis('equal')

# Add title
plt.title('Overall Timeout Effectiveness\n(Ability to Reduce Opponent Offensive Efficiency)', fontsize=16)
plt.savefig('timeout_effectiveness_pie.png', dpi=300, bbox_inches='tight')
plt.show()

# Create pie chart for run termination
plt.figure(figsize=(10, 10))

# Calculate termination statistics
terminated_count = int(df['run_terminated'].sum())
continued_count = len(df) - terminated_count
termination_rate = df['run_terminated'].mean()
continuation_rate = 1 - termination_rate

# Create data for the pie chart
labels = [f'Run Terminated\n{terminated_count} timeouts ({termination_rate:.1%})', 
         f'Run Continued\n{continued_count} timeouts ({continuation_rate:.1%})']
sizes = [termination_rate * 100, continuation_rate * 100]
colors = ['green', 'red']
explode = (0.1, 0)  # Explode the first slice

# Create pie chart
patches, texts, autotexts = plt.pie(sizes, explode=explode, labels=labels, colors=colors,
                                  autopct='%1.1f%%', shadow=True, startangle=90, 
                                  textprops={'fontsize': 14, 'fontweight': 'bold'})

# Equal aspect ratio ensures the pie chart is circular
plt.axis('equal')

# Add title
plt.title('Opponent Scoring Run Termination After Timeout', fontsize=16)
plt.savefig('run_termination_pie.png', dpi=300, bbox_inches='tight')
plt.show()

![Timeout Effectiveness Pie](DSA-210-PROJECT/dsa project/outputs/figures/timeout_effectiveness_pie.png)
![Run Termination Pie](DSA-210-PROJECT/dsa project/outputs/figures/run_termination_pie.png)

These pie charts provide a clear visual summary of our findings:

1. **Timeout Effectiveness**: 57.4% of timeouts effectively reduced opponent offensive efficiency, compared to 42.6% where efficiency increased or remained unchanged.

2. **Run Termination**: Similarly, 57.4% of opponent scoring runs were terminated after timeouts, while 42.6% continued despite the timeout.

These results provide empirical support for the coaching practice of calling timeouts to disrupt opponent momentum, with timeouts showing majority effectiveness by both metrics.

### 10. Summary of Findings

This analysis has explored the effectiveness of timeouts in disrupting opponent momentum during NBA games. Here are the key findings:

1. **Overall Effectiveness:** Timeouts are generally effective at disrupting opponent momentum, with 57.4% of timeouts resulting in decreased offensive efficiency. The average decrease in offensive efficiency was -3.655 points per possession, which is statistically significant.

2. **Shooting Efficiency:** Timeouts have a substantial impact on opponent shooting efficiency, with field goal percentage dropping from 55.9% to 41.2% and true shooting percentage dropping from 63.7% to 51.1% after timeouts.

3. **Timing Effects:** First quarter timeouts are particularly effective (66.1% effective), with effectiveness gradually decreasing as the game progresses (54.0% in the fourth quarter).

4. **Run Size Impact:** Timeouts appear more effective during larger scoring runs (76.1% effective for 8-9 point runs) compared to smaller runs (56.7% for 6-7 point runs).

5. **Team Variation:** Some teams (ORL, NOP, DET) demonstrate notably higher timeout effectiveness rates, suggesting variation in coaching strategies and team responsiveness.

6. **Consistency Across Eras:** Timeout effectiveness has remained relatively consistent across NBA seasons from 1999-2023, despite changes in playing style and pace.

### 11. Conclusions and Implications

Our analysis provides empirical evidence supporting the conventional basketball wisdom that timeouts can effectively disrupt opponent momentum. The consistent pattern of decreased offensive efficiency, particularly in shooting percentages, after timeouts suggests that coaches are justified in using timeouts strategically to halt opponent scoring runs.

**Practical Implications for Coaches:**

1. **Early Intervention:** First quarter timeouts show the highest effectiveness, suggesting coaches shouldn't hesitate to use timeouts early when opponents build momentum.

2. **Run Size Consideration:** Timeouts are particularly effective during larger scoring runs (8-9 points), indicating that coaches might want to prioritize timeouts at these critical junctures.

3. **Team-Specific Strategies:** The variation in team effectiveness suggests coaches should develop team-specific approaches to timeout usage, and potentially study what makes some teams more effective with their timeouts.

4. **Shooting Focus:** The dramatic decrease in shooting percentages suggests that a key focus during timeouts should be defensive adjustments specifically targeting the opponent's shooting opportunities.

**Limitations and Future Research:**

1. This analysis focused on NBA regular season games across five seasons. Future research could examine timeout effectiveness in playoff games, where pressure and preparation are heightened.

2. We did not distinguish between different types of timeouts (full vs. 20-second) or the specific content of timeout discussions.

3. Further research could explore additional contextual factors such as home court advantage, score differential, or specific matchup dynamics.

Overall, this analysis provides statistical validation for a long-standing coaching practice while offering nuanced insights into when and how timeouts are most effective at disrupting opponent momentum in NBA games.